In [23]:
import os,re,sys,math
import pandas as pd
import time
import datetime
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from scipy import stats
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import recall_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold  
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [24]:
train_path = 'data/dsjtzs_txfz_training.txt'
test_path = 'data/dsjtzs_txfz_test1.txt'
test_path_b = 'data/dsjtzs_txfz_testB.txt'

In [32]:

def inf_to_nan(x):
    for i in range(len(x)):
        if x[i] == float("inf") or x[i] == float("-inf"):
            x[i] = float('nan') # or x or return whatever makes sense
    return x


def sample_data(a,size = 0.7):
    np.random.seed(1)
    len_data = len(a)
    #size = np.random.random_sample()  #抽样比例随机
    size = int(size *len_data)
    if size < 1:
        size = 1
    index = np.sort((np.random.random(size)*len_data).astype(int))
    return np.array(a)[index]
def feature(path,extend=True):
    train = pd.read_csv(path,sep=' ',header=None,encoding='utf-8',names=['id','data','target','label'])
    train['data'] = train['data'].apply(lambda x:[list(map(float,point.split(','))) for point in x.split(';')[:-1]])

    
    if extend:
        #######扩充数据集#####


        train_new_1 = train.copy()
        train_new_1['data'] = train['data'].apply(lambda x :sample_data(x,size = 0.3))
        train_new_1['id'] = train['id'] + 3000 

        train=train.append(train_new_1)


        train_new_1 = train.copy()
        train_new_1['data'] = train['data'].apply(lambda x :sample_data(x,size = 0.7))
        train_new_1['id'] = train['id'] + 3000 
        train=train.append(train_new_1)
        
        

    

    train['target'] = train['target'].apply(lambda x: list(map(float,x.split(","))))
    train['data_x'] = train['data'].apply(lambda x: [i[0] for i in x ])
    train['data_y'] = train['data'].apply(lambda x: [i[1] for i in x ])
    train['data_t'] = train['data'].apply(lambda x: [i[2] for i in x ])
    train['data_x'] = train['data_x'].apply(lambda x: np.array(x))
    train['data_y'] = train['data_y'].apply(lambda x: np.array(x))
    train['data_t'] = train['data_t'].apply(lambda x: np.array(x))
    
    del train['data']

    
    train['target_x'] = train['target'].apply(lambda x: x[0])
    train['target_y'] = train['target'].apply(lambda x: x[1])
    #delt 
    train['delt_x'] = train['data_x'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_y'] = train['data_y'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_t'] = train['data_t'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_xy'] = train.delt_x**2 + train.delt_y**2
    train['delt_xy'] = train['delt_xy'].apply(lambda x: np.sqrt(x))
    #speed
    train['speed_x'] = (train.delt_x/train.delt_t).apply(lambda x :inf_to_nan(x)).apply(lambda x :np.nan_to_num(np.array(x)))
    train['speed_y'] = (train.delt_y/train.delt_t).apply(lambda x :inf_to_nan(x)).apply(lambda x :np.nan_to_num(np.array(x)))
    train['speed_xy'] = (train.delt_xy/train.delt_t).apply(lambda x :inf_to_nan(x)).apply(lambda x :np.nan_to_num(np.array(x)))

    #delt_speed
    train['delt_speed_x'] = train['speed_x'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_speed_y'] = train['speed_y'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_speed_xy'] = train['speed_xy'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_speed_t'] = train['delt_t'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])

    #acc
    train['acc_speed_x'] = (train.delt_speed_x/train.delt_speed_t).apply(lambda x :inf_to_nan(x)).apply(lambda x :np.nan_to_num(np.array(x)))
    train['acc_speed_y'] = (train.delt_speed_y/train.delt_speed_t).apply(lambda x :inf_to_nan(x)).apply(lambda x :np.nan_to_num(np.array(x)))
    train['acc_speed_xy'] = (train.delt_speed_xy/train.delt_speed_t).apply(lambda x :inf_to_nan(x)).apply(lambda x :np.nan_to_num(np.array(x)))

    #轨迹长度
    train['len_x'] =  train['data_x'].apply(lambda x: len(x))


    #first
    train['first_data_x'] = train['data_x'].apply(lambda x : x[0])
    train['first_speed_x'] = train['speed_x'].apply(lambda x: x[0] if x.shape[0] > 0 else 0)
    train['first_data_y'] = train['data_y'].apply(lambda x : x[0])
    train['first_delt_t'] = train['delt_t'].apply(lambda x: x[0] if len(x) > 0 else 0)
    
    
    
    #
     
    train['time_delta_min'] = train['delt_t'].map(lambda x: x.min() if x.shape[0] > 0 else 0)
    train['distance_deltas_max'] = train['delt_xy'].map(lambda x: x.max() if x.shape[0] > 0 else 0)
    train['median_speed'] = train['speed_x'].map(lambda x: np.median(x) if x.shape[0] > 0 else 0)
    train['xs_delta_var'] = train['delt_x'].map(lambda x: x.var() if x.shape[0] > 0 else 0)
    train['xs_delta_max'] = train['delt_x'].map(lambda x: x.max() if x.shape[0] > 0 else 0)
    train['x_min'] = train['data_x'].map(lambda x: x.min())
    train['y_min'] = train['data_y'].map(lambda x: x.min())
    
    
    #####################
       # X_max与X_target关系
    train['X_max'] = train['data_x'].map(lambda x: x.max() if x.shape[0] > 0 else 0)
    train['get_target'] = train['X_max'] - train['target_x']
    
    #特征75 76
    train['data_x_return'] = train['delt_x'].apply(lambda x: np.where(x<0)[0].shape[0]>0 if x.shape[0]>0 else False)
    # #y坐标唯一值个数的标准差比平均值
    train['data_y_unique_value'] = train['data_y'].map(
            lambda x: np.unique(x,return_counts=True) if x.shape[0] > 0 else 0)
    train['data_y_unique_value_stdmean'] = train['data_y_unique_value'].map(
        lambda x: x[1].std()/x[1].mean() if x != 0 else 0)
    del train['data_y_unique_value']
    
    
    train['dxspeed_mean'] = train["delt_speed_x"].apply(lambda x :np.mean(x) if x.shape[0] > 0 else 0)  #x方向速度差分的平均值
    train['dxspeed_std']  = train["delt_speed_x"].apply(lambda x :np.std(x) if x.shape[0] > 0 else 0)    #x方向速度差分的标准差
    train['dxspeed_median']  = train["delt_speed_x"].apply(lambda x :np.median(x) if x.shape[0] > 0 else 0)    
    #x方向速度全局标准差与最后N个标准差比值
    train['speed_xstd']  = train["speed_x"].apply(lambda x :np.std(x) if x.shape[0] > 0 else 0)    #x方向速度差分的标准差
    train['speed_x_laststd']  = train["speed_x"].apply(lambda x :np.std(x[-9:]) if x.shape[0] > 0 else 0)    #x方向速度差分的标准差
    train['speed_xstd_laststd'] = train['speed_xstd'] / train['speed_x_laststd']
    train['speed_xstd_laststd'] = train['speed_xstd_laststd'].apply(lambda x : float('nan') if(x==float('-inf') or x==float('inf')) else x)
    
    ###0717
    
    #delt_speed
    train['delt_acc_x'] = train['delt_speed_x'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    #train['delt_acc_y'] = train['delt_speed_y'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_acc_t'] = train['delt_speed_t'].apply(lambda x: np.array(x)[1:]-np.array(x)[:-1])
    train['delt_acc_speed_x'] = (train.delt_acc_x/train.delt_acc_t).apply(lambda x :inf_to_nan(x)).apply(lambda x :np.nan_to_num(np.array(x)))
    train['dxacc_mean'] = train["delt_acc_speed_x"].apply(lambda x :np.mean(x) if x.shape[0] > 0 else 0)  #x方向速度差分的平均值
    train['dxacc_std']  = train["delt_acc_speed_x"].apply(lambda x :np.std(x) if x.shape[0] > 0 else 0)    #x方向速度差分的标准差
    train['dxacc_median']  = train["delt_acc_speed_x"].apply(lambda x :np.median(x) if x.shape[0] > 0 else 0)    
    
    
    #0718 19
    from scipy import stats
    train['log_delt_x'] = train['data_x'].apply(lambda x: np.log1p(np.array(x)[1:]-np.array(x)[:-1]))
    train['log_delt_y'] = train['data_y'].apply(lambda x: np.log1p(np.array(x)[1:]-np.array(x)[:-1]))
    train['log_delt_t'] = train['data_t'].apply(lambda x: np.log1p(np.array(x)[1:]-np.array(x)[:-1]))
    
    train['xy_angles'] = train['log_delt_x'] - train['log_delt_y']
    train['xt_angles'] = train['log_delt_x'] - train['log_delt_t']
    train['xy_angles_kurtosis']= train['xy_angles'].apply(lambda x: stats.kurtosis(x))
    train['xt_angles_max'] =train['xt_angles'].apply(lambda x :np.max(x) if x.shape[0] > 0 else 0)  #x方向速度差分的平均值
    feats = ['id','label',
           # 'len_x',
    'first_data_x',
     'first_speed_x',
     'first_data_y',
     'first_delt_t',
     'time_delta_min',
     'distance_deltas_max',
     'median_speed',
     'xs_delta_var',
     'xs_delta_max',
     'x_min',
     'y_min',
     'X_max',
     'get_target',
     'data_x_return',
     'data_y_unique_value_stdmean',
     'dxspeed_mean',
     'dxspeed_std',
     'dxspeed_median',
           'speed_xstd',
     'speed_x_laststd',
     'speed_xstd_laststd',
             
     'dxacc_mean',    #715 feature
     'dxacc_std', #gooooood
     'xt_angles_max', #0718
     'xy_angles_kurtosis',#0719

             
        ]
    
    
    
    
    return train[feats]

In [33]:
def customedscore(preds, dtrain):
    label = dtrain.get_label()
    pred = [int(i>=0.50) for i in preds]
    confusion_matrixs = confusion_matrix(label, pred)
    recall =float(confusion_matrixs[0][0]) / float(confusion_matrixs[0][1]+confusion_matrixs[0][0])
    
    precision = float(confusion_matrixs[0][0]) / float(confusion_matrixs[1][0]+confusion_matrixs[0][0])
    F = 5*precision* recall/(2*precision+3*recall)*100
    return 'FSCORE',float(F)

def getscore(val_y, val_y_pre):
    confusion_matrixs = confusion_matrix(val_y, val_y_pre)
    recall =float(confusion_matrixs[0][0]) / float(confusion_matrixs[0][1]+confusion_matrixs[0][0])
    precision = float(confusion_matrixs[0][0]) / float(confusion_matrixs[1][0]+confusion_matrixs[0][0])
    # print(confusion_matrixs)
    # print("recall:",recall)
    # print("precision:",precision)
    F = 5*precision* recall/(2*precision+3*recall)*100
    # print("score :",F)
    return  recall , precision,F
def xgb_pre(params,train_x,train_y,val_x):
    import xgboost as xgb
    #######################xgboost###########
    trainset1 = xgb.DMatrix(train_x,label=train_y,missing=-100)
    dval_x = xgb.DMatrix(val_x,missing=-100)
    int_params = params["int_params"]
    num_boost_rounds = params["num_boost_rounds"]
    threshold = params["threshold"]
    model = xgb.train(int_params,trainset1,
        num_boost_round=num_boost_rounds,feval = customedscore,maximize=False)
    val_y_pre = model.predict(dval_x)
    val_y_pred = [int(i >= threshold) for i in val_y_pre]

    feature_score = model.get_fscore()
    feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
    print(feature_score)
    return val_y_pre,val_y_pred


def xgbcv(params,train_feature,train_label):
    recall,precision,score =[],[],[] 
    l1 =[]
    for i in range(10):
        train_x,val_x ,train_y,val_y=train_test_split(train_feature,train_label, test_size=0.8,random_state=i)
        val_y_pre, val_y_pred = xgb_pre(params,train_x,train_y,val_x)
        r,p,s = getscore(val_y, val_y_pred)
        recall.append(r);precision.append(p),score.append(s)
    print("recall mean    : %.5f  recall var     : %.4f"%(np.mean(recall),np.var(recall)))
    print("precision mean : %.5f  precision var  : %.5f"%(np.mean(precision),np.var(precision)))
    print("score mean     :%.5f  score var      :%.5f"%(np.mean(score),np.var(score)))
def xgbcv2(train_feature1,train_feature2,train_label):
    recall,precision,score =[],[],[] 
    l1 =[]
    for i in range(10):
        train_x1,val_x1 ,train_y,val_y=train_test_split(train_feature1,train_label, test_size=0.8,random_state=i)
        train_x2,val_x2 ,train_y,val_y=train_test_split(train_feature2,train_label, test_size=0.8,random_state=i)
        import xgboost as xgb
        #######################xgboost###########
        val_y_pre,val_y_pred = bagging(train_x1,val_x1 ,train_x2,val_x2 ,train_y)
        r,p,s = getscore(val_y, val_y_pred)
        recall.append(r);precision.append(p),score.append(s)
    print("recall mean    : %.5f  recall var     : %.4f"%(np.mean(recall),np.var(recall)))
    print("precision mean : %.5f  precision var  : %.5f"%(np.mean(precision),np.var(precision)))
    print("score mean     :%.5f  score var      :%.5f"%(np.mean(score),np.var(score)))


def bagging(train_feature1,test_feature1,train_feature2,test_feature2,train_label):
    threshold = 0.70
    t = 0.45
    val_y_pre1,val_y_pred1 = xgb_pre(params,train_feature1,train_label,test_feature1)
    val_y_pre2,val_y_pred2 = xgb_pre(params,train_feature2,train_label,test_feature2)
    val_y_pre =val_y_pre1*t+val_y_pre2*(1-t)
    val_y_pred = [int(i >= threshold) for i in val_y_pre]
    return val_y_pre, val_y_pred

def bagging2(train_feature1,test_feature1,train_feature2,test_feature2,train_label,train_label2):
    threshold = 0.65
    t = 0.7
    val_y_pre1,val_y_pred1 = xgb_pre(params,train_feature1,train_label,test_feature1)
    val_y_pre2,val_y_pred2 = xgb_pre(params,train_feature2,train_label2,test_feature2)
    val_y_pre =val_y_pre1*t+val_y_pre2*(1-t)
    val_y_pred = [int(i >= threshold) for i in val_y_pre]
    return val_y_pre, val_y_pred

In [34]:
#train = feature(train_path,extend = False)
train = feature(train_path,extend = True)

In [5]:

test = feature(test_path,extend = False)

In [6]:
#test_b = feature(test_path_b,extend = False)

In [35]:
feats2 = ['id','label',
  
'first_data_x', 'first_speed_x', 'first_data_y', 'first_delt_t', 'speed_xstd_laststd', 'X_max', 'get_target', #20+first

'dxspeed_mean', 'dxspeed_std', 'data_x_return', 'data_y_unique_value_stdmean',#特征75 76

 'time_delta_min', 'distance_deltas_max',  'median_speed', 'y_min', 'x_min', 'xs_delta_var', 'xs_delta_max',#开源部分特征
 'dxacc_mean',    #715 feature
          'dxacc_std', #gooooood
'xt_angles_max', #0718
     'xy_angles_kurtosis',#0719

        ]

In [36]:
final_train= train[feats2]
final_test= test[feats2]

In [1]:
# final_train= train[feats2]
# final_test= test_b[feats2]

In [38]:

final_train=final_train.fillna(-100)
final_test=final_test.fillna(-100)

In [39]:
params = {'threshold':0.65,'num_boost_rounds':150,
'int_params':{ 'silent': 1,  'objective': 'binary:logistic' , 
   # 'gamma':0.2,
       # 'min_child_weight':1,
        # 'max_depth':5,
        # 'lambda':10,
        # 'subsample':0.7,
        # 'colsample_bytree':0.7,
        # 'colsample_bylevel':0.7,
        # 'eta': 0.5,
         'tree_method':'exact',
        }}
train_data1 ,train_label = final_train.drop(['id','label'], axis=1).astype(float),train.label


xgbcv(params,train_data1,train_label)
test_data1 = final_test.drop(['id','label'], axis=1).astype(float)

final_test['label_prob'] = 0
final_test['label_prob'] ,final_test["label"]= xgb_pre(params,train_data1,train_label,test_data1)

print("negative number:",len(test)-final_test.label.sum())
final_test[final_test.label>0]['id'].to_csv("submission/{0}.txt".format(len(test)-final_test.label.sum()), header=None, index=False)
# final_test=final_test.sort_values(by='label_prob')
# final_test.iloc[0:20000].id.to_csv('submission/{0}_top2w.txt'.format(100000-final_test.label.sum()), header=None, index=False)
# final_test.iloc[20000:100000].id.to_csv("submission/{0}_8W.txt".format(100000-final_test.label.sum()), header=None, index=False)

[('first_delt_t', 70), ('xt_angles_max', 60), ('median_speed', 55), ('get_target', 48), ('x_min', 37), ('data_y_unique_value_stdmean', 32), ('first_data_x', 29), ('time_delta_min', 27), ('dxspeed_std', 27), ('y_min', 24), ('first_data_y', 20), ('distance_deltas_max', 19), ('speed_xstd_laststd', 19), ('X_max', 18), ('xy_angles_kurtosis', 17), ('dxacc_mean', 16), ('first_speed_x', 15), ('dxspeed_mean', 11), ('xs_delta_max', 11), ('dxacc_std', 10), ('xs_delta_var', 6)]
[('first_data_y', 43), ('first_delt_t', 43), ('xt_angles_max', 41), ('dxspeed_mean', 37), ('dxspeed_std', 37), ('first_data_x', 33), ('data_y_unique_value_stdmean', 30), ('time_delta_min', 30), ('dxacc_std', 29), ('get_target', 28), ('first_speed_x', 27), ('median_speed', 27), ('x_min', 22), ('dxacc_mean', 17), ('xs_delta_var', 13), ('speed_xstd_laststd', 12), ('xy_angles_kurtosis', 12), ('distance_deltas_max', 11), ('xs_delta_max', 11), ('X_max', 5), ('y_min', 3)]
[('first_delt_t', 54), ('x_min', 45), ('xt_angles_max', 44)